In [1]:
import pickle
import numpy as np
import pandas as pd
from data_process import Get_Offense
from class_offfense import Offense
import distance_utils
import time
from tqdm import tqdm
import json


import importlib
importlib.reload(distance_utils)
from distance_utils import *

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def get_data():
    with open('data/final_data.pkl', 'rb') as file:
        loaded_data = pickle.load(file)
    list_offense =[]
    for o in loaded_data:
        list_offense.append(Offense(o))

    return(list_offense)

In [3]:
list_offense = get_data()
len(list_offense)

67820

In [4]:
max_time = -1
for o in list_offense:
    o.list_coords = o.list_coords[:8]
    o.list_action_type = o.list_action_type[:8]
    o.list_time = o.list_time[:8]
    m_time = o.list_time[-1]
    if m_time > max_time:
        max_time = m_time

max_time

248

# cluster


In [5]:
for i, offense in enumerate(list_offense):
    offense.index_in_current_list = i

print(list_offense[5].index_in_current_list)

5


# create distance metric

## DTW

In [7]:
num_offenses = len(list_offense)
#num_offenses = 100
score_dict_dtw = {}

for i in tqdm(range(num_offenses), desc="Processing", ncols=100):
    #for i in range(num_offenses):
    score_dict_dtw[str(i)] = {}
    score_dict_dtw[str(i)][str(i)] = 0
    for j in range(i + 1, num_offenses):
        cost = DTW(list_offense[i], list_offense[j])
        score_dict_dtw[str(i)][str(j)] = cost
       

# Print the score_dict
print("Score Dictionary:")
print(score_dict_dtw)


Processing: 100%|█████████████████████████████████████████████████| 100/100 [00:02<00:00, 37.81it/s]

Score Dictionary:
{'0': {'0': 0, '1': 0.12636102266505114, '2': 0.08317203205148953, '3': 0.10554409518304897, '4': 0.10103402935920493, '5': 0.09602008675387522, '6': 0.1673071508390704, '7': 0.11173750332734778, '8': 0.09383706468897196, '9': 0.05273669457034463, '10': 0.18113530173949804, '11': 0.08809323676095906, '12': 0.18279939232164882, '13': 0.09275099361224874, '14': 0.14175738676462066, '15': 0.10371521097761285, '16': 0.09239807919599041, '17': 0.09996036229571123, '18': 0.08537372962783125, '19': 0.06871564893973822, '20': 0.11148061646825501, '21': 0.07679324927804504, '22': 0.10061417758165081, '23': 0.07089685903407783, '24': 0.08842770176743986, '25': 0.07836743502973435, '26': 0.12468630888276827, '27': 0.12301388811297966, '28': 0.07411987210868931, '29': 0.07411987210868931, '30': 0.0549203873692877, '31': 0.17870563106376486, '32': 0.11620249791066037, '33': 0.08677952599805781, '34': 0.16275922378182836, '35': 0.16970382520491176, '36': 0.1636136444391719, '37': 0

In [8]:
score_dict_dtw.keys()

dict_keys(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99'])

In [ ]:
with open('data/Score_dtw.json', 'w') as file:
    json.dump(score_dict_dtw, file, indent = 4)

In [9]:
def DTW_for_model(index1, index2):
    try:
        i1 = str(int(index1[0]))
        i2 = str(int(index2[0]))
    except:
        i1 = str(int(index1))
        i2 = str(int(index2))
    try:
        return score_dict_dtw[i1][i2]
    except:
        return score_dict_dtw[i2][i1]

In [10]:
all_index = [i for i in range(len(list_offense))]
#all_index = [i for i in range(100)]

global_centroid_dtw  = calculate_centroid(all_index , DTW_for_model)
global_centroid_dtw

95

## O3D

In [11]:
num_offenses = len(list_offense)
#num_offenses = 100
score_dict_o3d = {}
for i in tqdm(range(num_offenses), desc="Processing", ncols=100):
    #for i in range(num_offenses):
    score_dict_o3d[str(i)] = {}
    score_dict_o3d[str(i)][str(i)] = 0
    for j in range(i + 1, num_offenses):
        cost = O3D(list_offense[i], list_offense[j])
        score_dict_o3d[str(i)][str(j)] = cost
        pass
       

# Print the score_dict
print("Score Dictionary:")
print(score_dict_o3d)

Processing: 100%|█████████████████████████████████████████████████| 100/100 [00:01<00:00, 83.46it/s]

Score Dictionary:
{'0': {'0': 0, '1': 0.11097896295597702, '2': 0.14294620949394513, '3': 0.21943366277055848, '4': 0.175605678022044, '5': 0.22849083947168858, '6': 0.15422887627349166, '7': 0.2518768164240247, '8': 0.19396329547923982, '9': 0.10365919503373218, '10': 0.14969086780544955, '11': 0.1997820545272061, '12': 0.13331304801753519, '13': 0.15179856968392458, '14': 0.3206825000807989, '15': 0.24325396029029261, '16': 0.163970171997245, '17': 0.20224266192615153, '18': 0.16122998508226905, '19': 0.09612496259306291, '20': 0.1251861360959611, '21': 0.11077622838678342, '22': 0.07842002751888394, '23': 0.1878971353819457, '24': 0.22378834081341542, '25': 0.16603434068926412, '26': 0.1371042597444389, '27': 0.2849014737988972, '28': 0.13285346541002102, '29': 0.13285346541002102, '30': 0.08034877799158098, '31': 0.14413116509600424, '32': 0.26376980681585166, '33': 0.08353090693113645, '34': 0.11048050958205236, '35': 0.15790594258985324, '36': 0.09352874949621127, '37': 0.1092944

In [ ]:
with open('data/Score_o3d.json', 'w') as file:
    json.dump(score_dict_o3d, file, indent = 4)

In [12]:
def O3D_for_model(index1, index2):
    try:
        i1 = str(int(index1[0]))
        i2 = str(int(index2[0]))
    except:
        i1 = str(int(index1))
        i2 = str(int(index2))
    try:
        return score_dict_o3d[i1][i2]
    except:
        return score_dict_o3d[i2][i1]

In [14]:
all_index = [i for i in range(len(list_offense))]
#all_index = [i for i in range(100)]

global_centroid_o3d = calculate_centroid(all_index , O3D_for_model)
global_centroid_o3d


95

# Clustering

## DBSCAN

In [102]:
from sklearn.cluster import DBSCAN


### TDW

In [266]:
data = np.array([offense.index_in_current_list for offense in list_offense])
data_reshaped = data.reshape(-1, 1)
#data_reshaped = data_reshaped[:100]

eps_list = [0.01 , 0.02 ,0.03 , 0.04 , 0.05 , 0.06 , 0.07]
min_samples_list = [1,2,3,4,5,6,7,8]
results = []

for e in eps_list:
    for s in min_samples_list:
        dbscan = DBSCAN(metric= DTW_for_model, eps=e, min_samples=s)
        labels_per_index = dbscan.fit_predict(data_reshaped)
        labels_per_cluster = convert_to_clusters(labels_per_index)
        score = calculate_bcss_wcss_ratio(labels_per_cluster[:-1], DTW_for_model, global_centroid_dtw)
        outliers = len(labels_per_cluster[-1])
        num_clusters = len(labels_per_cluster) - 1
        
        # Append results to the list
        results.append({
            "Epsilon": e,
            "Min Samples": s,
            "Score": score,
            "Outliers": outliers,
            "Num Clusters": num_clusters
        })

# Create a DataFrame from the results list
results_df = pd.DataFrame(results)


'\nfor o, l in zip(list_offense, cluster_labels):\n    o.DBSCAN_DTW = l'

In [267]:
results_df

,Epsilon,Min Samples,Score,Outliers,Num Clusters
0,0.01,1,204.709297,0,86
1,0.01,2,60.402714,82,4
2,0.01,3,51.221290,86,2
3,0.01,4,60.850000,87,2
4,0.01,5,73.589084,91,1
5,0.01,6,73.589084,91,1
6,0.01,7,73.589084,91,1
7,0.02,1,68.488699,0,80
8,0.02,2,27.735152,74,6
9,0.02,3,31.103139,80,3


### O3D

In [276]:
data = np.array([offense.index_in_current_list for offense in list_offense])
data_reshaped = data.reshape(-1, 1)
data_reshaped = data_reshaped[:100]

eps_list = [0.01 , 0.02 ,0.03 , 0.04 , 0.05 , 0.06 , 0.07]
min_samples_list = [1,2,3,4,5,6,7,8]
results = []

for e in eps_list:
    for s in min_samples_list:
        dbscan = DBSCAN(metric= O3D_for_model, eps=e, min_samples=s)
        labels_per_index = dbscan.fit_predict(data_reshaped)
        labels_per_cluster = convert_to_clusters(labels_per_index)
        score = calculate_bcss_wcss_ratio(labels_per_cluster[:-1], O3D_for_model, global_centroid_o3d)
        outliers = len(labels_per_cluster[-1])
        num_clusters = len(labels_per_cluster) - 1
        
        # Append results to the list
        results.append({
            "Epsilon": e,
            "Min Samples": s,
            "Score": score,
            "Outliers": outliers,
            "Num Clusters": num_clusters
        })

# Create a DataFrame from the results list
results_df = pd.DataFrame(results)


In [277]:
results_df

,Epsilon,Min Samples,Score,Outliers,Num Clusters
0,0.01,1,598.991526,0,95
1,0.01,2,71.512900,90,5
2,0.01,3,inf,100,0
3,0.01,4,inf,100,0
4,0.01,5,inf,100,0
5,0.01,6,inf,100,0
6,0.01,7,inf,100,0
7,0.01,8,inf,100,0
8,0.02,1,79.327989,0,86
9,0.02,2,19.658906,80,6


## K-means

## DTW

In [314]:
data = np.array([offense.index_in_current_list for offense in list_offense])
data = data[:100]

max_iterations = 300
results = []
for n in range(4,15):
    n_clusters = n
    labels, centroids = custom_kmeans(data, n_clusters, max_iterations, DTW_for_model)
    label_mapping = {label: i for i, label in enumerate(centroids)}
    labels_per_index = [label_mapping[l] for l in labels]
    labels_per_cluster = convert_to_clusters(labels_per_index)
    score = calculate_bcss_wcss_ratio(labels_per_cluster, DTW_for_model, global_centroid_dtw)
    num_clusters = len(centroids) 
        
    # Append results to the list
    results.append({
        "Score": score,
        "Num Clusters": num_clusters
    })

results_df = pd.DataFrame(results)




[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95
 96 97 98 99]


In [315]:
results_df

,Score,Num Clusters
0,1.945026,4
1,2.064881,5
2,2.288889,6
3,2.211113,7
4,2.273854,8
5,2.784905,9
6,2.482044,10
7,2.557306,11
8,2.940917,12
9,2.776421,13


## O3D

In [316]:
data = np.array([offense.index_in_current_list for offense in list_offense])
data = data[:100]

max_iterations = 300
results = []
for n in range(4,15):
    n_clusters = n
    labels, centroids = custom_kmeans(data, n_clusters, max_iterations, O3D_for_model)
    label_mapping = {label: i for i, label in enumerate(centroids)}
    labels_per_index = [label_mapping[l] for l in labels]
    labels_per_cluster = convert_to_clusters(labels_per_index)
    score = calculate_bcss_wcss_ratio(labels_per_cluster, O3D_for_model, global_centroid_o3d)
    num_clusters = len(centroids) 
        
    # Append results to the list
    results.append({
        "Score": score,
        "Num Clusters": num_clusters
    })

results_df = pd.DataFrame(results)

In [317]:
results_df

,Score,Num Clusters
0,1.802969,4
1,1.916403,5
2,1.664436,6
3,2.080446,7
4,2.159661,8
5,2.118849,9
6,2.206545,10
7,2.404682,11
8,2.321697,12
9,2.501543,13
